In [2]:
using CairoMakie, Dierckx, QuantEcon, LinearAlgebra, Optim

In [10]:
pars = (;r = 0.04, # interest rate
    β = 0.9, # discount factor
    γ = 1.0, # risk aversion
    nz = 19, # number of grid points for z
    ρ = 0.9, # persistence of AR1
    μ = 0.0, # mean of AR1
    σ = 0.003, # std of AR1
    na = 31, # number of asset grid points
    θ = 4.0, # grid expansion factor
    toler = 4e-7, # tolerance
    maxiter = 1000, # maximum no. of iterations
    lb = 0.0, # lower bound of capital grid
    ub = 100.0) # upper bound of capital grid

(r = 0.04, β = 0.9, γ = 1.0, nz = 19, ρ = 0.9, μ = 0.0, σ = 0.003, na = 31, θ = 4.0, toler = 4.0e-7, maxiter = 1000, lb = 0.0, ub = 100.0)

In [17]:
function utility(c, pars)
    (; γ) = pars
    if γ == 1.0
        return log(c)
    else
        return c^(1-γ)/(1-γ)
    end
end

function ar1(pars)
    (;ρ, μ, σ, nz) = pars
    mc = QuantEcon.rouwenhorst(nz, μ, ρ, σ)
    return mc.p, mc.state_values
end

function exp_grid(pars)
    (; na, θ, lb, ub) = pars
    grid = LinRange(0.0,1.0,na)
    exp_grid = lb .+ (ub - lb) .* grid.^θ
    return exp_grid
end

function resources(Avals, Zvals, j, i, pars)
    (; r) = pars
    return (1+r)*Avals[j] + exp(Zvals[i])
end

function interpV(Avals, v_slice, pars)
    interp_v = Spline1D(Avals, v_slice, k=3, bc="extrapolate")
    return interp_v
end

function optimise(Avals, Zvals, v_init, v_new, policy, Π, pars)
    (; β, na, nz, lb, ub) = pars
    for i in 1:nz
        expected_value = v_init * Π[i,:]
        interpolation = interpV(Avals, expected_value, pars)
        for j in 1:na
            obj(ap) = - (utility(resources(Avals, Zvals, j, i, pars) - ap, pars) + β * interpolation(ap))
            ub = resources(Avals, Zvals, j, i, pars)  
            res = optimize(obj, lb, ub)
            policy[j,i] = res.minimizer
            v_new[j,i] = -res.minimum
        end
    end
    return v_new, policy
end

optimise (generic function with 1 method)

In [18]:
function vfi(v_init, pars)
    (; maxiter, toler, nz, na) = pars
    Π, Zvals = ar1(pars)
    Avals = exp_grid(pars)
    v_new = similar(v_init)
    policy = similar(v_init)
    error = toler + 1
    iter = 0
    if iter == 0
        println("Iterating...")
    end
    while ((error > toler) && (iter < maxiter))
        v_new, policy = optimise(Avals, Zvals, v_init, v_new, policy, Π, pars)
        error = maximum(abs.(v_new - v_init) ./ (1 .+ abs.(v_new)))
        v_init .= v_new
        if iter % 10 == 0
            println("--------------------")
            println("Iteration: $iter, Error: $error")
        end
        iter += 1
    end
    println("--------------------")
    println("Converged in $iter iterations")
    println("--------------------")
    return v_new, policy
end

v_init = ones(pars.na, pars.nz)

@time begin
v_out, pol_out = vfi(v_init, pars)
end

Iterating...
--------------------
Iteration: 0, Error: 1.0
--------------------
Iteration: 10, Error: 0.056620934285226784
--------------------
Iteration: 20, Error: 0.02077951046236053
--------------------
Iteration: 30, Error: 0.007458940374677626
--------------------
Iteration: 40, Error: 0.002566977218584129
--------------------
Iteration: 50, Error: 0.0008818118346576934
--------------------
Iteration: 60, Error: 0.0003058885301452547
--------------------
Iteration: 70, Error: 0.00010646586038091435
--------------------
Iteration: 80, Error: 3.709919763745958e-5
--------------------
Iteration: 90, Error: 1.2932877852399941e-5
--------------------
Iteration: 100, Error: 4.5090738379743e-6
--------------------
Iteration: 110, Error: 1.5721752765085055e-6
--------------------
Iteration: 120, Error: 5.481785708229686e-7
--------------------
Converged in 124 iterations
--------------------
  0.681632 seconds (456.06 k allocations: 50.092 MiB, 1.75% gc time, 59.67% compilation time)


([-1.8858597292329349 -1.4615956605209999 … 11.742371003760274 14.28518163601281; -1.8800483022476209 -1.4577897105552562 … 11.742394976347963 14.285199638578943; … ; 19.115558389152305 19.116533602636583 … 21.339553521315906 22.35164457167154; 20.161008539711595 20.16188265442984 … 22.19004592170976 23.15179441523146], [3.5181593885267114e-16 3.200489815799136e-16 … 24.521800780372 38.534996581720705; 3.551605967068761e-16 3.221160938145513e-16 … 24.52191399137461 38.53511024666447; … ; 78.89635212295346 78.90677923975633 … 105.72726133356818 120.30417724944164; 90.70426886351585 90.71470140473004 … 117.86716512271046 132.9640995694287])